# Benchmark of methods for brain tumor segmentation 

In [2]:
import warnings
warnings.filterwarnings('ignore')

import nibabel as nib
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import numpy as np
import cv2 as cv
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from skfuzzy.cluster import cmeans
from dipy.segment.mask import median_otsu
from time import time
import glob
%matplotlib inline

N_ITERS = 10

# Extracting and preprocessing 

In [9]:
NUMPASS = 5
MEDIAN_RADIUS = 11
HIT = 0.25
BLUR_RADIUS = 3


N_CLUSTERS = 5
N_COMPONENTS = 2

M = 2.0
EPS = 0.01
MAX_IT = 100
    
K1 = np.ones((3, 3), np.uint16)

IMG_ROOT = '/Users/macbook/Documents/Education/4_Year/DiplomaWork/Images/brain/TCIA Low grade glioma/nifti/'

def get_data(root, patients):
    for patient in iter(patients):
        img_path = root + 'LGG-%s/LGG-%s_T2.nii.gz' % (patient[0], patient[0])
        seg_path = root + 'LGG-%s/LGG-%s-Segmentation.nii.gz' % (patient[0], patient[0])
        img = nib.load(img_path).get_data()[:, :, patient[1]]
        seg = nib.load(seg_path).get_data()[:, :, patient[1]]
        yield (img, seg)
    
def draw(img, size=(8, 8), cmap='jet'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.plot()
    


# from random import shuffle

# def get_data():
#     NUM = 0
#     for img_path, seg_path in zip(glob.glob(IMG_ROOT + '/**/*T2.nii.gz'), glob.glob(IMG_ROOT + '/**/*Segmentation.nii.gz')):
#         img = nib.load(img_path).get_data()
#         seg = nib.load(seg_path).get_data()
#         s = np.sum(seg.reshape((seg.shape[2], -1)).astype(np.int32), axis=1)
#         indicies = s > 0
#         LEN = indicies.shape[0]
#         NUM += LEN
#         indicies = indicies.reshape((-1))
#         if NUM > LIMIT:
#             raise StopIteration
#         else:
#             print(s)
#             indicies = np.argwhere(indicies).reshape(-1)
#             print(indicies)
#             img2 = img[:, :, indicies]
            
#             seg2 = seg[:, :, indicies]
            
#             print(seg2.shape)
#             print(np.sum(seg2.reshape((seg2.shape[2], -1)).astype(np.int32), axis=1))
#             yield img2[:, :, 4], seg2[:, :, 4]
        

        
def preprocess(data):
    for data_ in iter(data):
        img = data_[0].astype(np.float32)
        img = cv.medianBlur(img, 3)
        img, _ = median_otsu(img, numpass=NUMPASS, median_radius=MEDIAN_RADIUS)
        original_shape = img.shape
        img = ((img - np.min(img)) / (np.max(img) - np.min(img))).astype(np.float32)
        blurred = cv.blur(img, (19, 19))
        edges = np.clip(img - blurred, 0.0, 1.0)
        edges[edges > HIT] = 1.0
        edges[edges <= HIT] = 0.0
        edges = cv.dilate(edges, np.ones((2, 2)), iterations=1)
        img = np.clip(img - edges, 0.0, 1.0)
        yield (img, data_[1])
    
PATIENTS = [
    ('104', 38),  
#     ('220', 25),
#     ('225', 35),
#     ('229', 14),
#     ('249', 35),
#     ('374', 45),
#     ('600', 25),
#     ('500', 33),
#     ('547', 39),
#     ('311', 39),
#     (357, 35),
#     (387, 16),
#     (345, 30),
#     (346, 48),
#     (351, 24),
#     (354, 28),
#     (355, 40),
#     (357, 36),
#     (359, 33),
#     (360, 24),
#     (361, 19),
#     (363, 23),
#     (365, 28),
#     (367, 28),
#     (371, 35),
#     (373, 34),
#     (374, 44),
#     (375, 43),
#     (377, 38),
#     (380, 37),
#     (383, 38),
#     (385, 36),
#     (388, 43),
#     (391, 25),
#     (394, 38),
#     (396, 37),
#     (492, 40),
#     (500, 34),
#     (506, 26),
#     (515, 25),
#     (516, 43),
#     (518, 49),
#     (519, 23),
    (520, 33),
] 

N = len(PATIENTS)
print(N)

DATA = list(get_data(IMG_ROOT, PATIENTS))
PREP = list(preprocess(DATA))

# INDEX = 0
# orig = DATA[INDEX][0]
# # image = PREP[INDEX][0]
# gt = DATA[INDEX][1]

# # res, labels = multivariate_kmeans(image)
# # print('Dice (%)', calc_dice(res, gt))
# # draw(orig)
# # # draw(labels)
# # # draw(res)
# # draw(gt)

# plt.figure(figsize=(8, 8))
# plt.imshow(orig, 'gray')
# plt.imshow(gt, alpha=0.3)

# plt.plot()

2


# K Means

In [4]:
def kmeans(img, n_clusters):
    img = cv.blur(img, (15, 15))
    x = img.reshape((-1, 1))
    model = KMeans(n_clusters=n_clusters, random_state=0, algorithm='full', precompute_distances=False, n_init=1)
    model.fit(x)
    c_index = np.argmax(model.cluster_centers_.reshape((-1)))
    flat = np.full(img.shape[0] * img.shape[1], 0, dtype=np.uint8)
    flat[model.labels_ == c_index] = 1
    mask = flat.reshape(img.shape)
    mask = cv.dilate(mask, K1, iterations=1)
    mask = cv.erode(mask, K1, iterations=1)
    return mask

# Weighted Multivariate K Means

In [ ]:
def multivariate_kmeans(img, n_clusters):
    f1 = cv.blur(img, (7, 7)).reshape((-1, 1))
    f2 = cv.blur(img, (9, 9)).reshape((-1, 1)) 
    f3 = cv.blur(img, (5, 5)).reshape((-1, 1)) 
    f4 = img.reshape((-1, 1)) * 0.45
    xc = np.linspace(0, 1.0, img.shape[1])
    yc = np.linspace(0, 1.0, img.shape[0])
    f5 = np.transpose([np.tile(xc, len(xc)), np.repeat(yc, len(yc))]).reshape((-1, 2)) * 0.015
    f3 = cv.blur(img, (5, 5)).reshape((-1, 1)) 
    X = np.concatenate((f1, f2, f3, f4, f5), axis=1)
    x = X
    model = KMeans(n_clusters=n_clusters, random_state=0, algorithm='full', precompute_distances=False, n_init=1)
    model.fit(x)
    c_index = np.argmax(model.cluster_centers_[:,3])
    flat = np.full(img.shape[0] * img.shape[1], 0, dtype=np.uint8)
    flat[model.labels_ == c_index] = 1
    mask = flat.reshape(img.shape)
    mask = cv.erode(mask, K1, iterations=1)
    mask = cv.dilate(mask, K1, iterations=1)
    return mask

# Fuzzy C Means

In [ ]:
def fcm(img, n_clusters):
    M = 2
    img = cv.blur(img, (15, 15))
    flat = img.reshape((1, -1))
    c, u, a1, a2, a3, a4, a5 = cmeans(flat, n_clusters, M, EPS, 50)
    tumor_index = np.argmax(c, axis=0)
    defuz = np.argmax(u, axis=0)
    mask = np.full(defuz.shape[0], 0, dtype=np.uint16)
    mask[defuz == tumor_index] = 1
    mask = mask.reshape(img.shape)
    mask = cv.erode(mask, K1, iterations=1)
    mask = cv.dilate(mask, K1, iterations=1)
    return mask

# Gaussian Mixture

In [ ]:
def gaussian_mixture(img, n_components):
    img = cv.blur(img, (15, 15))
    x = img.reshape((-1, 1))
    model = GaussianMixture(n_components=n_components, covariance_type='spherical')
    model.fit(x)
    labels = model.predict(x)
    c_index = np.argmax(model.means_)
    flat = np.full(img.shape[0] * img.shape[1], 0, dtype=np.uint8)
    flat[labels == c_index] = 1
    mask = flat.reshape(img.shape)
    mask = cv.erode(mask, K1, iterations=1)
    mask = cv.dilate(mask, K1, iterations=1)
    return mask

# Binary thresholding

In [ ]:
def find_threshold_cpu(img, max_it=100, eps=0.001):
    cols = img.shape[1]

    mid = cols // 2
    p1 = img[:, :mid].reshape((-1))
    p2 = img[:, mid:].reshape((-1))
    img = img.reshape((-1))

    it = -1
    while True:
        t1 = np.mean(p1)
        t2 = np.mean(p2)

        t = (t1 + t2) / 2.0
        it += 1
        if it >= max_it or abs(t1 - t2) <= eps:
            return t
        else:
            p1 = img[img < t]
            p2 = img[img >= t]
        
def threshold(img):
    t = find_threshold_cpu(img, max_it=MAX_IT, eps=EPS)
    mask = (img > t).astype(np.int16)
    mask = cv.erode(mask, K1, iterations=1)
    mask = cv.dilate(mask, K1, iterations=1)
    return mask

In [5]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import cv2 as cv
import numpy as np
from dipy.segment.mask import median_otsu
from pydicom import Dataset

from jinja2 import Template
import numpy as np

BLOCKDIM = 1024

SRC = '''
    #define N {{N}}
    #define BLOCKDIM 1024
    
    struct Cluster{
        float sum;
        int count;
    };
    
    __device__ Cluster clusters_d[(N + BLOCKDIM - 1) / BLOCKDIM];
    
    __device__ float euclidian_dist(const float a, const float b){
        float dist = a - b;
        return hypotf(dist, dist);
    }
    
    __global__ void relabel(const float* src, const float* clusters, int n, int nClusters, int* labels){
        int pos = threadIdx.x + blockIdx.x * blockDim.x;
        if(pos < n){
            float minDist = 1.0f;
            int clusterIndex = 0;
            for(int c = 0; c < nClusters; c++){
                float dist = euclidian_dist(src[pos], clusters[c]);
                if(dist <= minDist){
                    clusterIndex = c;
                    minDist = dist;
                }
            }
            labels[pos] = clusterIndex;
        }
    }
    
    __global__ void calculateClusters(const float* src, const int* labels, int n, int clusterIndex){
        extern __shared__ Cluster _clusters[];
        int pos = threadIdx.x + blockIdx.x * blockDim.x;
        int tid = threadIdx.x;
        _clusters[tid] = Cluster();
        _clusters[tid].sum = 0.0f;
        _clusters[tid].count = 0;
        if(pos < n && labels[pos] == clusterIndex){
            _clusters[tid].sum = src[pos];
            _clusters[tid].count = 1;
        }
        __syncthreads();
        for(unsigned int stride = blockDim.x / 2; stride > 0; stride /= 2){
            if(threadIdx.x < stride){
                _clusters[tid].sum += _clusters[tid + stride].sum;
            }
            __syncthreads();
            if(threadIdx.x < stride){
                _clusters[tid].count += _clusters[tid + stride].count;
            }
            __syncthreads();
        }
        __syncthreads();
        if(threadIdx.x == 0){
            clusters_d[blockIdx.x].sum = _clusters[0].sum;
            clusters_d[blockIdx.x].count = _clusters[0].count;
        }
    }
    
    __global__ void findCenters(int n, int clusterIndex, float* dst){
        extern __shared__ Cluster _clusters[];
        int pos = threadIdx.x + blockIdx.x * blockDim.x;
        int tid = threadIdx.x;
        _clusters[tid] = clusters_d[pos];
        __syncthreads();
        for(unsigned int stride = blockDim.x / 2; stride > 0; stride /= 2){
            if(tid < stride){
                _clusters[tid].sum += _clusters[tid + stride].sum;
            }
            __syncthreads();
            if(tid < stride){
                _clusters[tid].count += _clusters[tid + stride].count;
            }
            __syncthreads();
        }
        __syncthreads();
        if(tid == 0){
            dst[clusterIndex] = _clusters[0].count > 0 ? _clusters[0].sum / (_clusters[0].count * 1.0f) : 0.0f;
        }
    }
'''

def cuda_kmeans(img, **kwargs):
        w = img.shape[1]
        h = img.shape[0]
        n = w * h
        max_it = 30
        n_clusters = kwargs.get('n_clusters', 3)
        blur_radius = kwargs.get('blur_radius', 5)

        img = cv.blur(img, (13, 13))
        src = img.reshape((-1))

        module = SourceModule(Template(SRC).render(N=n))
        relabel = module.get_function('relabel')
        calculate_clusters = module.get_function('calculateClusters')
        find_centers = module.get_function('findCenters')

        t0 = time() * 1000
        centers = np.random.rand(n_clusters).astype(np.float32)

        # Image
        src_gpu = cuda.mem_alloc(src.nbytes)
        cuda.memcpy_htod(src_gpu, src)

        # Cluster centers

        centers_gpu = cuda.mem_alloc(centers.nbytes)
        cuda.memcpy_htod(centers_gpu, centers)

        # Labels
        labels = np.empty_like(src).astype(np.int32)
        labels_gpu = cuda.mem_alloc(labels.nbytes)
        cuda.memcpy_htod(labels_gpu, labels)
        
        start = cuda.Event()
        end = cuda.Event()
        
        t1 = time() * 1000
        
        start.record()
        for it in range(max_it):
            relabel(src_gpu, centers_gpu, np.int32(n), np.int32(n_clusters), labels_gpu,
                    block=(BLOCKDIM, 1, 1), grid=((n + BLOCKDIM - 1) // BLOCKDIM, 1))
            for c in range(n_clusters):
                calculate_clusters(src_gpu, labels_gpu, np.int32(n), np.int32(c),
                                   block=(BLOCKDIM, 1, 1), grid=((n + BLOCKDIM - 1) // BLOCKDIM, 1),
                                   shared=8 * BLOCKDIM)
                find_centers(np.int32(n), np.int32(c), centers_gpu,
                             block=((n + BLOCKDIM - 1) // BLOCKDIM, 1, 1), grid=((1, 1)),
                             shared=8 * (n + BLOCKDIM - 1) // BLOCKDIM)

        
        end.record()
        end.synchronize()
        msecs = end.time_since(start)*1e-3
        msecs *= 1000
        msecs += t1 - t0
        
        t0 = time() * 1000
        cuda.memcpy_dtoh(labels, labels_gpu)
        cuda.memcpy_dtoh(centers, centers_gpu)
#
        labels = labels.reshape((-1))
        c_index = np.argmax(centers)
        flat = np.full(n, 0, dtype=np.uint8)
        flat[labels == c_index] = 1
        mask = flat.reshape((h, w))
        mask = cv.erode(mask, kernel=K1, iterations=1)
        mask = cv.dilate(mask, kernel=K1, iterations=1)
        t1 = time() * 1000
        msecs += t1 - t0
        return mask, msecs

cuda_kmeans.is_cuda = True

# Profile average time and accuracy (Dice-Sørensen) per image

In [10]:
# ALGS = {
#     'K Means': kmeans,
# #     'Threshold': threshold,
#     'Gaussian Mixture': gaussian_mixture,
# #     'Fuzzy C Means': fcm,
# #     'Multivariate K Means': multivariate_kmeans
# }

import collections
ALGS = collections.OrderedDict()
ALGS['K Means'] = kmeans
# ALGS['Fuzzy C Means'] = fcm
# ALGS['Gaussian Mixture'] = gaussian_mixture
# ALGS['Multivariate K Means'] = multivariate_kmeans
ALGS['CUDA K Means'] = cuda_kmeans


def t(alpha, gl):
    return scipy.stats.t.ppf(1-(alpha/2), gl)

TP_INDEX = 0
FP_INDEX = 1
FN_INDEX = 2
DC_INDEX = 3
PR_INDEX = 4
RC_INDEX = 5
F1_INDEX = 6

def calc_accuracy(seg, gt):
    size = seg.shape[0] * seg.shape[1]
    tp = np.sum(seg[gt==1])
    tn = np.sum(seg[gt==0])
    
    fp = np.sum((gt==0) & (seg==1))
    fn = np.sum((gt==1) & (seg==0))
    dc = 2.0 * tp / (np.sum(gt) + np.sum(seg)) * 100.0
    
#     dc = 2 * tp / (2 * tp + fp + fn) * 100.0
#     precision = tp / (tp + fp) 
#     recall = tp / (tp + fn)
#     f1 = 2 * precision * recall / (precision + recall) * 100.0

    precision = tp / (tp + fp) * 100.0 
    recall = tp / (tp + fn) * 100.0

    tp = tp * 100.0 / size
    fp = fp * 100.0 / size
    fn = fn * 100.0 / size
    return dc

def profile(alg, data, n_clusters):
    avg_time = []
    avg_accuracy = []
    for data_item in iter(data):
        time_it = []
        accuracy_it = []
        img, gt = data_item[0], data_item[1]
        for i in range(N_ITERS):
            duration = 0.0
            if getattr(alg, 'is_cuda', False):
                seg, duration = alg(img)
            else:
                t0 = time()
                seg = alg(img, n_clusters)
                t1 = time()
                duration = (t1 - t0) * 1000.0
            
            time_it.append([duration])
            accuracy_it.append(calc_accuracy(seg, gt))
        avg_time.append(np.mean(time_it))
        avg_accuracy.append(np.mean(accuracy_it))
    return avg_time, avg_accuracy

# def profile_cuda(alg, data):
#     avg_time = []
#     avg_accuracy = []
#     for data_item in iter(data):
#         time_it = np.asarray([])
#         accuracy_it = np.empty((1, 7))
#         img, gt = data_item[0], data_item[1]
#         for i in range(N_ITERS):
            
#             time_it.append([(t1 - t0) * 1000.0])
#             accuracy_it.append(calc_accuracy(seg, gt))
#         avg_time.append(np.mean(time_it))
#         avg_accuracy.append(np.mean(accuracy_it, axis=0))
#     return avg_time, avg_accuracy

In [ ]:
N_CLUSTERS_RANGE = (3, 5)
t_avg_accuracy = []
conf95_accuracy = []

t_avg_time = []
conf95_time = []

CONF_INT_COEFF = 1.0 / sqrt(N) * t(0.05, N - 1)

avg_time_per_n_clusters = []
avg_acc_per_n_clusters = []

for N_CLUSTERS in range(N_CLUSTERS_RANGE[0], N_CLUSTERS_RANGE[1]):
    print('================================')
    print('N CLUSTERS = ', N_CLUSTERS)
    print('================================')
    time_t, accuracy_t = {}, {}
    for alg in ALGS.keys():
        avg_time, avg_accuracy = profile(ALGS[alg], PREP, N_CLUSTERS)
        time_t[alg] = avg_time
        accuracy_t[alg] = avg_accuracy

#     print('Average time per image')
#     print(time_t)
#     print('\n')

    print('Average accuracy per image')
    print(accuracy_t)
    print('\n')

    
    t_avg_time_alg = []
    t_avg_accuracy_alg = []
    conf95_time_alg = []
    conf95_accuracy_alg = []
    
    avg_time_per_alg = []
    for alg, time_per_image in time_t.items():
        avg_time_per_alg.append(time_per_image)
        t_avg_time_ = np.mean(time_per_image)
        t_std_time_ = np.std(time_per_image)
        t_avg_time_alg.append(t_avg_time_)
#         t_std_time[i].append(t_std_time_)
        conf95_time_alg.append(
            [
                t_avg_time_ - t_std_time_ * CONF_INT_COEFF, 
                t_avg_time_ + t_std_time_ * CONF_INT_COEFF
            ]
        )
    avg_time_per_n_clusters.append(avg_time_per_alg)

    avg_acc_per_alg = []
    for alg, accuracy_per_image in accuracy_t.items():
        avg_acc_per_alg.append(accuracy_per_image)
        t_avg_accuracy_ = np.mean(accuracy_per_image)
        t_std_accuracy_ = np.std(accuracy_per_image)
        t_avg_accuracy_alg.append(t_avg_accuracy_)
#         t_std_accuracy[i].append(t_std_accuracy_)
        conf95_accuracy_alg.append(
            [
                t_avg_accuracy_ - t_std_accuracy_ * CONF_INT_COEFF, 
                t_avg_accuracy_ + t_std_accuracy_ * CONF_INT_COEFF
            ]
        )
    avg_acc_per_n_clusters.append(avg_time_per_alg)
    
    t_avg_time.append(t_avg_time_alg)
    t_avg_accuracy.append(t_avg_accuracy_alg)
    conf95_time.append(conf95_time_alg)
    conf95_accuracy.append(conf95_accuracy_alg)

print('Total average time (msecs)')
print(t_avg_time)
print('\n')

# print('Total std of time (msecs)^2')
# print(t_std_time)
# print('\n')

print('95% Confidence interval of time (msecs)')
print(conf95_time)
print('\n')

print('Total average accuracy (%)')
print(t_avg_accuracy)
print('\n')

#     print('Total std of accuracy (%)')
#     print(t_std_accuracy)
#     print('\n')

print('95% Confidence interval of accuracy (%)')
print(conf95_accuracy)
print('\n')

N CLUSTERS =  3
Average accuracy per image
{'K Means': [85.30834340991535, 86.71742808798646], 'CUDA K Means': [85.09732360097324, 85.92150170648463]}


N CLUSTERS =  4


In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10

res_time = np.asarray(t_avg_time)
res_acc = np.asarray(t_avg_accuracy)
conf95_time_res = np.asarray(conf95_time)
conf95_acc_res = np.asarray(conf95_accuracy)

avg_time_per_n_clusters = np.asarray(avg_time_per_n_clusters)
avg_acc_per_n_clusters = np.asarray(avg_acc_per_n_clusters)

X = np.arange(N_CLUSTERS_RANGE[0], N_CLUSTERS_RANGE[1])
plt.title('Average time (msecs)')
plt.xticks(X)
plt.xlabel('n clusters')
plt.ylabel('msecs')
# plt.yscale('log', basey=2)
for i, alg in zip(range(len(ALGS.keys())), ALGS.keys()):
    # ax.set_xticklabels(['K=%d' % k for k in range(3, 6)])
    plt.plot(X, res_time[:,i], label=alg)
#     plt.fill_between(X, conf95_time_res[:,i,0], conf95_time_res[:,i,1], alpha=0.15)
    # plt.axis('equal')
plt.legend()
plt.show()


plt.title('Average accuracy (%)')
plt.xticks(X)
plt.xlabel('n clusters')
plt.ylabel('%')
for i, alg in zip(range(len(ALGS.keys())), ALGS.keys()):
    # ax.set_xticklabels(['K=%d' % k for k in range(3, 6)])
    plt.plot(X, res_acc[:,i], label=alg)
#     plt.fill_between(X, conf95_acc_res[:,i,0], conf95_acc_res[:,i,1], alpha=0.15)
    # plt.axis('equal')
plt.legend()
plt.show()

import matplotlib
palegreen = matplotlib.colors.colorConverter.to_rgb('#8CFF6F')
paleblue = matplotlib.colors.colorConverter.to_rgb('#708DFF')
    
for i, alg in zip(range(len(ALGS.keys())), ALGS.keys()):
    fig = plt.figure(1, figsize=(9, 9))
    ax = fig.add_subplot(111)
    
    ax.set_title('%s - Time (msecs)' % alg)
    bp = ax.boxplot(avg_time_per_n_clusters[:,i,:].T)
    ax.set_xticklabels(list(range(N_CLUSTERS_RANGE[0], N_CLUSTERS_RANGE[1])))
    plt.show()

    fig = plt.figure(1, figsize=(9, 9))
    ax = fig.add_subplot(111)
    
    ax.set_title('%s - Accuracy (%%)' % alg)
    bp = ax.boxplot(avg_acc_per_n_clusters[:,i,:].T)
    ax.set_xticklabels(list(range(N_CLUSTERS_RANGE[0], N_CLUSTERS_RANGE[1])))
    plt.show()

for i, alg in zip(range(len(ALGS.keys())), ALGS.keys()):
    print("\n")
    print("===========================")
    print('----->', alg, '<-----')
    print("\n")
    print('[Average time]')
    print(res_time[:,i])
    print("\n")
    print('[Average accuracy]')
    print(res_acc[:,i])
    print("\n")
    print('[Conf 95% time]')
    print(conf95_time_res[:,i,:])
    print("\n")
    print('[Conf 95% accuracy]')
    print(conf95_acc_res[:,i,:])


# Profile CUDA based algorithms (KMeans)

In [ ]:
# print('Profile cuda')
# CUDA_ALGS = {
#     'CUDA K Means': cuda_kmeans
# }
# time_t, accuracy_t = {}, {}
# for alg in CUDA_ALGS.keys():
#     avg_time, avg_accuracy = profile_cuda(CUDA_ALGS[alg], PREP)
#     time_t[alg] = avg_time
#     accuracy_t[alg] = avg_accuracy

# print('Average time per image')
# print(pd.DataFrame.from_dict(time_t))
# print('\n')

# print('Average accuracy per image')
# print(pd.DataFrame.from_dict(accuracy_t))
# print('\n')

# avg_time = {}
# std_time = {}
# conf95_time = {}

# avg_accuracy = {}
# std_accuracy = {}
# conf95_accuracy = {}

# CONF_INT_COEFF = 1.0 / sqrt(N) * t(0.05, N - 1)

# for alg, time_per_image in time_t.items():
#     avg_time[alg] = np.mean(time_per_image)
#     std_time[alg] = np.std(time_per_image)
#     conf95_time[alg] = [
#         avg_time[alg] - std_time[alg] * CONF_INT_COEFF,
#         avg_time[alg] + std_time[alg] * CONF_INT_COEFF,
#     ]
    
# for alg, accuracy_per_image in accuracy_t.items():
#     avg_accuracy[alg] = np.mean(accuracy_per_image)
#     std_accuracy[alg] = np.std(accuracy_per_image)
#     conf95_accuracy[alg] = [
#         avg_accuracy[alg] - std_accuracy[alg] * CONF_INT_COEFF,
#         avg_accuracy[alg] + std_accuracy[alg] * CONF_INT_COEFF
#     ]

# print('Total average time (msecs)')
# print(pd.DataFrame.from_dict(avg_time, orient='index'))
# print('\n')

# print('Total std of time (msecs)^2')
# print(pd.DataFrame.from_dict(std_time, orient='index'))
# print('\n')

# print('95% Confidence interval of time (msecs)')
# print(pd.DataFrame.from_dict(conf95_time, orient='index'))
# print('\n')

# print('Total average accuracy (%)')
# print(pd.DataFrame.from_dict(avg_accuracy, orient='index'))
# print('\n')

# print('Total std of accuracy (%)')
# print(pd.DataFrame.from_dict(std_accuracy, orient='index'))
# print('\n')

# print('95% Confidence interval of accuracy (%)')
# print(pd.DataFrame.from_dict(conf95_accuracy, orient='index'))
# print('\n')

In [ ]:
# from dipy.segment.mask import otsu
# INDEX = 8
# img = PREP[INDEX][0]
# seg = PREP[INDEX][1]
# draw(img)
# draw(kmeans(img, 3))
# draw(seg)

In [ ]:
from medpy.io import load
image_data, image_header = load('/Users/macbook/Documents/Education/4_Year/DiplomaWork/Images/brain/BRATS Glioblastoma/BRATS2015_TRAINING/LGG/brats_2013_pat0001_1/VSD.Brain.XX.O.MR_T2.54635/VSD.Brain.XX.O.MR_T2.54635.mha')
draw(image_data)